# Сложные объединения
---

Вы уже умеете присоединять строки друг к другу путём добавления столбцов «сбоку» — с помощью различных видов `JOIN`.

А что если нам необходимо присоединить несколько результатов «снизу», так, чтобы получить общий результат в выводе?

Ответу на этот вопрос и посвящён текущий модуль.

## (1) Знакомство с данными
---

В данном блоке мы будем работать с данными о компании, организующей перевозки грузов.

Интересующие нас данные хранятся в таблицах `city`, `customer`, `driver`, `shipment`, `truck`. Давайте внимательно их изучим.

Ниже представлена *ER*-диаграмма (от англ. *entity-relation*, дословно — «сущность-связь»), которая отображает существующие связи между отдельными таблицами.

![](data/dst3-u2-md4_1_1.jpg)

Таблица `city` — это справочник городов. Структура справочника представлена ниже.

| НАЗВАНИЕ ПОЛЯ | ТИП ДАННЫХ |                     ОПИСАНИЕ                    |
|:-------------:|:----------:|:-----------------------------------------------:|
| city_id       | integer    | уникальный идентификатор города, первичный ключ |
| city_name     | text       | название города                                 |
| state         | text       | штат, к которому относится город                |
| population    | integer    | население города                                |
| area          | numeric    | площадь города                                  |

Таблица `customer` — это справочник клиентов. У компании, с данными которой мы работаем, только корпоративные клиенты, поэтому в таблице нет привычных данных о возрасте и поле. Справочник содержит следующие поля:

|  НАЗВАНИЕ ПОЛЯ | ТИП ДАННЫХ |                      ОПИСАНИЕ                     |
|:--------------:|:----------:|:-------------------------------------------------:|
| cust_id        | integer    | уникальный идентификатор клиента, первичный ключ  |
| cust_name      | text       | название клиента                                  |
| annual_revenue | numeric    | ежегодная выручка                                 |
| cust_type      | text       | тип пользователя                                  |
| address        | text       | адрес                                             |
| zip            | integer    | почтовый индекс                                   |
| phone          | text       | телефон                                           |
| city_id        | integer    | идентификатор города, внешний ключ к таблице city |

Следующая таблица — `driver` — справочник водителей. Перечень сведений, содержащихся в таблице, представлен ниже.

| НАЗВАНИЕ ПОЛЯ | ТИП ДАННЫХ |                          ОПИСАНИЕ                          |
|:-------------:|:----------:|:----------------------------------------------------------:|
| driver_id     | integer    | уникальный идентификатор водителя, первичный ключ          |
| first_name    | text       | имя водителя                                               |
| last_name     | text       | фамилия водителя                                           |
| address       | text       | адрес водителя                                             |
| zip_code      | integer    | почтовый индекс водителя                                   |
| phone         | text       | телефон водителя                                           |
| city_id       | integer    | идентификатор города водителя, внешний ключ к таблице city |

В таблице `truck` хранится информация о грузовиках, на которых осуществляются перевозки. Данные о них представлены в следующем виде:

| НАЗВАНИЕ ПОЛЯ | ТИП ДАННЫХ |                      ОПИСАНИЕ                      |
|:-------------:|:----------:|:--------------------------------------------------:|
| truck_id      | integer    | Уникальный идентификатор грузовика, первичный ключ |
| make          | text       | Производитель грузовика                            |
| model_year    | integer    | Дата выпуска грузовика                             |

Последняя таблица в датасете, `shipment`, — таблица с данными непосредственно о доставках. Она описывает взаимодействие всех перечисленных сущностей, а потому содержит наибольшее количество ссылок на другие таблицы.

| НАЗВАНИЕ ПОЛЯ | ТИП ДАННЫХ |                                        ОПИСАНИЕ                                       |
|:-------------:|:----------:|:-------------------------------------------------------------------------------------:|
| ship_id       | integer    | уникальный идентификатор доставки, первичный ключ                                     |
| cust_id       | integer    | идентификатор клиента, которому отправлена доставка, внешний ключ к таблице customer  |
| weight        | numeric    | вес посылки                                                                           |
| truck_id      | integer    | идентификатор грузовика, на котором отправлена доставка, внешний ключ к таблице truck |
| driver_id     | integer    | идентификатор водителя, который осуществлял доставку, внешний ключ к таблице driver   |
| city_id       | integer    | идентификатор города в который совершена доставка, внешний ключ к таблице city        |
| ship_date     | date       | дата доставки                                                                         |

### Задание 1.1

> Укажите название города с максимальным весом единичной доставки.


```sql
SELECT
    city.city_name
FROM
    sql.shipment AS shipment
    JOIN sql.city AS city ON shipment.city_id = city.city_id
GROUP BY
    city.city_id,
    shipment.weight
ORDER BY
    shipment.weight DESC
LIMIT
    1
```

**Ответ:** Green Bay



> Сколько различных производителей грузовиков перечислено в таблице `truck`?


```sql
SELECT
    COUNT(DISTINCT truck.make)
FROM
    sql.truck AS truck
```

**Ответ:** 3

> Как зовут водителя (`first_name`), который совершил наибольшее количество доставок одному клиенту?


```sql
SELECT
    CONCAT(driver.first_name, ' ', driver.last_name)
FROM
    sql.shipment AS shipment
    JOIN sql.driver AS driver ON shipment.driver_id = driver.driver_id
GROUP BY
    shipment.cust_id,
    shipment.driver_id,
    driver.first_name,
    driver.last_name
ORDER BY
    COUNT(shipment.driver_id) DESC
LIMIT
    1
```

**Ответ:** Holger (Nohr)

> Укажите даты первой и последней по времени доставок в таблице `shipment`.
>
> Ответ введите в формате ДД.ММ.ГГГГ.


```sql
SELECT
    shipment.ship_date
FROM
    sql.shipment AS shipment
ORDER BY
    shipment.ship_date ASC
LIMIT
    1
```

Ответ: 08.01.2016 27.12.2017

> Укажите имя клиента, получившего наибольшее количество доставок за 2017 год.


```sql
SELECT
    customer.cust_name
FROM
    sql.shipment AS shipment
    JOIN sql.customer AS customer ON shipment.cust_id = customer.cust_id
GROUP BY
    shipment.cust_id,
    customer.cust_name
ORDER BY
    COUNT(shipment.cust_id) DESC
LIMIT
    1
```

**Ответ:** Autoware In

## (2-6) UNION

---

Вернёмся к центральному вопросу модуля: как соединить несколько результатов, чтобы получить в выводе один общий?

### (2) UNION. Принцип и условия работы.
---

Чтобы разобраться в этом вопросе, смоделируем ситуацию.

Допустим, мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм.

Для этого напишем простой запрос:


```sql
SELECT
    book_name object_name,
    'книга' object_description
FROM
    public.books
UNION
ALL
SELECT
    movie_title,
    'фильм'
FROM
    sql.kinopoisk
```

Визуально произведённое нами действие можно представить следующим образом:

![](data/dst3-u2-md4_2_1.png)

Общий принцип мы поняли, разберёмся в деталях:

В запросе мы использовали оператор UNION ALL — он присоединяет любой результат запроса к другому «снизу» при условии, что у них **одинаковая структура**, а именно:

* одинаковый тип данных;

    ![](data/dst3-u2-md4_2_2.png)

* одинаковое количество столбцов;

    ![](data/dst3-u2-md4_2_3.png)

* одинаковый порядок столбцов согласно типу данных.

    ![](data/dst3-u2-md4_2_4.png)

**Виды `UNION`**

Оператор присоединения существует в двух вариантах:

* `UNION` выводит только уникальные записи;
* `UNION ALL` присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности.

**Важно!** `UNION` оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с `DISTINCT`). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать `UNION ALL`.


**Синтаксис**


```sql
SELECT
    n columns
FROM
    table_1
UNION
ALL
SELECT
    n columns
FROM
    table_2...
UNION
ALL
SELECT
    n columns
FROM
    table_n
```

Пришла пора испытать функцию `UNION ALL`  на практике.

Обратимся к нашему датасету о транспортной компании и посмотрим, как сформировать справочник с ID всех таблиц и указанием объекта, к которому он относится.


```sql
SELECT
         c.city_id object_name,  'id города' object_type /*выбираем колонку city_id и задаём ей алиас object_name, сами задаём объект 'id города' и название столбца object_type*/
FROM 
         sql.city c /*из схемы sql и таблицы city, задаём алиас таблице — с*/
UNION ALL /*оператор присоединения*/
SELECT
         d.driver_id other_name,  'id водителя' other_type /*выбираем колонку driver_id и задаём ей алиас other_name, сами задаём объект 'id водителя' и название столбца other_type*/
FROM 
         sql.driver d  /*из схемы sql и таблицы driver, задаём алиас таблице — d*/
UNION ALL /*оператор присоединения*/
SELECT
         s.ship_id,  'id доставки' /*выбираем колонку ship_id, сами задаём объект 'id доставки'*/
FROM 
         sql.shipment s /*из схемы sql и таблицы shipment, задаём алиас таблице — s*/
UNION ALL /*оператор присоединения*/
SELECT
         c.cust_id,  'id клиента' /*выбираем колонку cust_id, сами задаём объект 'id клиента'*/
FROM 
         sql.customer c /*из схемы sql и таблицы customer, задаём алиас таблице — c*/
UNION ALL /*оператор присоединения*/
SELECT
         t.truck_id,  'id грузовика' /*выбираем колонку truck_id, сами задаём объект 'id грузовика'*/
FROM 
         sql.truck t /*из схемы sql и таблицы truck, задаём алиас таблице — t*/
ORDER BY 1 /*сортировка по первому столбцу*/
```

**Обратите внимание!** Несмотря на исходные названия колонок *other_name* и *other_type* во втором подзапросе, в выводе мы получим названия, которые дали в первом блоке: *object_name* и *object_type*.

Другая особенность — в применении сортировки `ORDER BY`: она всегда будет относиться к итоговому результату всего запроса с `UNION ALL`.

В случаях, когда необходимо применить команду `ORDER BY` или `LIMIT` не к итоговому результату, а к каждой части запроса, можно обернуть подзапросы в скобки.

Чтобы посмотреть, как это работает, вернёмся к нашему примеру с общим справочником по фильмам и книгам.

Мы уже знаем, что можно легко и непринуждённо применить операторы `ORDER BY` и `LIMIT` ко всему результату запроса.


```sql
SELECT
    book_name object_name,
    'книга' object_descritption
FROM
    public.books
UNION
ALL
SELECT
    movie_title,
    'фильм'
FROM
    sql.kinopoisk
ORDER BY
    1
LIMIT
    1
```

Всё бы хорошо, только в таком случае отсортирован будет весь общий справочник, а в выводе останется одна строка с названием объекта, идущим первым по алфавиту.

А если мы не хотим общую сортировку? Может, нам нужны строки с названием как фильма, так и книги, идущих первыми по алфавиту.

Нет ничего проще — отсортируем каждую часть запроса по отдельности и объединим результаты!

Просто добавим ORDER BY и LIMIT ещё и в первую часть запроса:


```sql
SELECT
    book_name object_name,
    'книга' object_descritption
FROM
    public.books
ORDER BY
    1
LIMIT
    1
UNION
ALL
SELECT
    movie_title,
    'фильм'
FROM
    sql.kinopoisk
ORDER BY
    1
LIMIT
    1
```


Вместо результата получим сообщение о синтаксической ошибке: *"...syntax error at or near "UNION"..."* Очевидно, этот фокус не удался.

Не стоит огорчаться, ведь проблему можно решить одним (ну, почти) движением руки — просто добавив скобки вокруг каждой из частей запроса.

```sql
(
    SELECT
        book_name object_name,
        'книга' object_descritption
    FROM
        public.books
    ORDER BY
        1
    LIMIT
        1
)
UNION
ALL (
    SELECT
        movie_title,
        'фильм'
    FROM
        sql.kinopoisk
    ORDER BY
        1
    LIMIT
        1
)
```

#### Задание 2.1

> Напишите запрос, который создает уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков. Результатом запроса должны быть два столбца: название и тип объекта (`city`, `state`, `driver`, `truck`). Отсортируйте список по названию объекта, а затем — по типу.


```sql
SELECT
    DISTINCT city.city_name AS object_name,
    'city' AS object_type
FROM
    sql.city AS city
UNION
ALL
SELECT
    DISTINCT city.state,
    'state'
FROM
    sql.city AS city
UNION
ALL
SELECT
    DISTINCT driver.first_name,
    'driver'
FROM
    sql.driver AS driver
UNION
ALL
SELECT
    DISTINCT truck.make,
    'truck'
FROM
    sql.truck AS truck
ORDER BY
    object_name,
    object_type
```

**Ошибка в формулировке:** не уникальный справочник, а справочник уникальных значений из таблиц:

**Эталонное решение**


```sql
SELECT
    c.city_name "название",
    'city' "тип объекта"
FROM
    sql.city c
UNION
SELECT
    c.state,
    'state'
FROM
    sql.city c
UNION
SELECT
    d.first_name,
    'driver'
FROM
    sql.driver d
UNION
SELECT
    t.make,
    'truck'
FROM
    sql.truck t
ORDER BY
    1,
    2
```

#### Задание 2.2

> Напишите запрос, который соберёт имена всех упомянутых городов и штатов из таблицы `city`. Результатом запроса должен быть один столбец `object_name`, отсортированный в алфавитном порядке.


```sql
SELECT
    city_name AS object_name
FROM
    sql.city
UNION
ALL
SELECT
    state
FROM
    sql.city
ORDER BY
    object_name
```

#### Задание 2.3

> Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов. Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов. Результатом запроса должен быть один столбец `object_name`, отсортированный в алфавитном порядке.


```sql
SELECT
    DISTINCT cities_states.object_name
FROM
    (
        SELECT
            city_name AS object_name
        FROM
            sql.city
        UNION
        ALL
        SELECT
            state
        FROM
            sql.city
        ORDER BY
            object_name
    ) AS cities_states
```

**!!Эталонное решение!!**


```sql
SELECT
    c.city_name object_name
FROM
    sql.city c
UNION
SELECT
    c.state
FROM
    sql.city c
ORDER BY
    1
```

### (3) UINION и ограничение типов данных
---

**Почему так важен тип данных?**

Как мы уже знаем, `UNION` может быть использован только в случае полного соответствия столбцов и их типов в объединяемых запросах.

Допустим, мы хотим вывести список всех `id` городов и их названий в одном столбце.

Давайте напишем запрос, который позволит получить нужный нам результат.


```sql
SELECT
    c.city_id
FROM
    sql.city c
UNION
ALL
SELECT
    cc.city_name
FROM
    sql.city cc
```

Вместо результата вы получите сообщение об ошибке: *"ERROR: UNION types integer and text cannot be matched"*. Дело в том, что мы попытались объединить числовой и 
строковый типы в одной колонке, а это невозможно.

Если мы всё же хотим выполнить поставленную задачу, придётся привести оба столбца к одному типу данных. Не каждый текст может быть приведён к числу, зато каждое число может быть представлено в текстовом формате.

Забегая вперёд, скажем пару слов о типизации столбцов. Для типизации в `Postgres` составляется запрос по модели `column_name::column_type`.

Таким образом, чтобы перевести `city_id` в текст, нам потребуется написать `city_id::text`.

**Важно!** Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для соединения разнородных сущностей. Главное — помнить, что сортировка текста отличается от сортировки чисел и дат.


Немного подправим запрос, чтобы получить желаемый результат.


```sql
SELECT
    c.city_id :: text
FROM
    sql.city c
UNION
ALL
SELECT
    cc.city_name
FROM
    sql.city cc
```

#### Задание 3.1

> Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник `contact`. Также добавьте столбец с именем водителя `first_name` и столбец `contact_type` с типом контакта (`phone` или `zip` в зависимости от типа). Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя.


```sql
SELECT
    phone AS contact,
    first_name,
    'phone' AS contact_type
FROM
    sql.driver
UNION
ALL
SELECT
    zip_code :: text,
    first_name,
    'zip'
FROM
    sql.driver
ORDER BY
    contact,
    first_name
```

**Эталонное решение**


```sql
SELECT
    d.zip_code :: text contact,
    d.first_name first_name,
    'zip' contact_type
FROM
    sql.driver d
UNION
SELECT
    dd.phone contact,
    dd.first_name first_name,
    'phone' contact_type
FROM
    sql.driver dd
ORDER BY
    1,
    2
```

### (4) UNION ALL и промежуточные итоги
---

**Возможности UNION**

Помимо соединения разнородных сущностей в единый справочник, `UNION ALL` часто используется для подведения промежуточных итогов и выведения результатов агрегатных функций.

Кроме агрегатных функций, в запросах с `UNION` могут использоваться функции группировки и выборки.


```sql
SELECT
    c.city_name,
    c.population
FROM
    sql.city c
UNION
ALL
SELECT
    'total',
    SUM(c.population)
FROM
    sql.city c
ORDER BY
    2 DESC
```

Визуально это действие можно представить так:

![](data/dst3-u2-md4_4_1.png)

Выполнив запрос в Metabase, вы заметите, что сначала выведен итоговый результат с суммой населения всех городов в таблице, а затем по каждому городу в отдельности.

#### Задание 4.1

> Напишите запрос, который выводит общее число доставок `total_shipments`, а также количество доставок в каждый день. Необходимые столбцы: `date_period`, `cnt_shipment`. Не забывайте о единой типизации. Упорядочите по убыванию столбца `date_period`.


```sql
SELECT
    dates_cnt.date_period :: text,
    dates_cnt.cnt_shipment
FROM
    (
        SELECT
            ship_date AS date_period,
            COUNT(ship_id) AS cnt_shipment
        FROM
            sql.shipment
        GROUP BY
            ship_date
        ORDER BY
            date_period DESC
    ) AS dates_cnt
UNION
ALL
SELECT
    'total',
    COUNT(ship_id)
FROM
    sql.shipment
```

**Эталонное решение**

```sql
SELECT
    s.ship_date :: text date_period,
    COUNT(*) cnt_shipment
FROM
    sql.shipment s
GROUP BY
    1
UNION
ALL
SELECT
    'total_shipments',
    COUNT(*)
FROM
    sql.shipment s
ORDER BY
    1 desc
```

### (5) UNION и дополнительные условия
---

`UNION` также может быть использован для разделения существующей выборки по критерию «выполнение определённого условия».

Например, с помощью `UNION` можно отобразить, у кого из водителей заполнен столбец с номером телефона.


```sql
SELECT
    d.first_name,
    d.last_name,
    'телефон заполнен' phone_info
FROM
    sql.driver d
WHERE
    d.phone IS NOT NULL
UNION
SELECT
    d.first_name,
    d.last_name,
    'телефон не заполнен' phone_info
FROM
    sql.driver d
WHERE
    d.phone IS NULL
```

#### Задание 5.1

> Напишите запрос, который выведет все города и штаты, в которых они расположены, а также информацию о том, была ли осуществлена доставка в этот город:
>
> * если в город была осуществлена доставка, то выводим 'доставка осуществлялась';
> * если нет — выводим 'доставка не осуществлялась'.
>
> Столбцы к выводу: `city_name`, `state`, `shipping_status`. Отсортируйте в алфавитном порядке по городу, а затем — по штату.

**Решение 1** (переусложненный)


```sql
SELECT
    shipment_not_completed.city_name,
    shipment_not_completed.state,
    'доставка не осуществлялась' AS shipment_info
FROM
    (
        SELECT
            city.city_name,
            city.state,
            shipment.ship_id
        FROM
            sql.city AS city
            LEFT JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
        GROUP BY
            city.city_id,
            shipment.ship_id
        HAVING
            shipment.ship_id IS NULL
    ) AS shipment_not_completed
UNION
ALL
SELECT
    city.city_name,
    city.state,
    'доставка осуществлялась'
FROM
    sql.city AS city
    JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
GROUP BY
    city.city_id
ORDER BY
    city_name,
    state
```

**Эталонное решение**


```sql
SELECT
    c.city_name AS city_name,
    c.state AS state,
    'доставка осуществлялась' AS shipping_status
FROM
    sql.city c
    LEFT JOIN sql.shipment s ON c.city_id = s.city_id
WHERE
    s.city_id IS NOT NULL
UNION
SELECT
    c.city_name AS city_name,
    c.state AS state,
    'доставка не осуществлялась' AS shipping_status
FROM
    sql.city c
    LEFT JOIN sql.shipment s ON c.city_id = s.city_id
WHERE
    s.city_id IS NULL
ORDER BY
    1,
    2
```

**Решение 2** (оптимизированный эталон)

```sql
SELECT
    city.city_name AS city_name,
    city.state AS state,
    'доставка осуществлялась' AS shipping_status
FROM
    sql.city AS city
    JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
UNION
SELECT
    city.city_name,
    city.state,
    'доставка не осуществлялась' AS shipping_status
FROM
    sql.city AS city
    LEFT JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
WHERE
    shipment.city_id IS NULL
ORDER BY
    city_name,
    state
```


**Решение 3** (вариант 1 исправленный)

```sql
SELECT
    city.city_name,
    city.state,
    'доставка не осуществлялась' AS shipment_info
FROM
    sql.city AS city
    LEFT JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
WHERE
    shipment.ship_id IS NULL
UNION
ALL
SELECT
    city.city_name,
    city.state,
    'доставка осуществлялась'
FROM
    sql.city AS city
    JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
GROUP BY
    city.city_id
ORDER BY
    city_name,
    state
```

#### Задание 5.2

> Напишите запрос, который выводит два столбца: `city_name` и `shippings_fake`. Выведите города, куда совершались доставки. Пусть первый столбец содержит название города, а второй формируется так:
> 
> * если в городе было более десяти доставок, вывести количество доставок в этот город как есть;
> * иначе — вывести количество доставок, увеличенное на пять.
>
> Отсортируйте по убыванию получившегося «нечестного» количества доставок, а затем — по имени в алфавитном порядке.

**Решение** (моё решение совпадает с эталонным)

```sql
SELECT
    city.city_name,
    COUNT(shipment.ship_id) AS shippings_fake
FROM
    sql.city AS city
    JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
GROUP BY
    city.city_id
HAVING
    COUNT(shipment.ship_id) > 10
UNION
ALL
SELECT
    city.city_name,
    COUNT(shipment.ship_id) + 5 AS shippings_fake
FROM
    sql.city AS city
    JOIN sql.shipment AS shipment ON city.city_id = shipment.city_id
GROUP BY
    city.city_id
HAVING
    COUNT(shipment.ship_id) <= 10
ORDER BY
    shippings_fake DESC,
    city_name
```

### (6) UNION и ручная генерация
---

`UNION` можно использовать для создания справочников прямо в коде запроса. К примеру, если мы хотим вручную ввести какие-то значения и произвести с ними некоторые манипуляции или дополнить существующую выдачу своими значениями.

Составим запрос, который позволит вывести первые три буквы алфавита и их порядковые номера.


```sql
SELECT
    'a' letter,
    '1' ordinal_position
UNION
SELECT
    'b',
    '2'
UNION
SELECT
    'c',
    '3'
```

Существуют сложные алгоритмы сравнения текстовых значений, но главный смысл сводится к одному: сравнение производится на основе таблицы `unicode` и позиции элемента в ней с учётом определённых условий.

#### Задание 6.1

> Напишите запрос, который выберет наибольшее из значений:
> 
> * 1000000;
> * 541;
> * -500;
> * 100.
> 
> Столбец с результатом назовите `result`.

**Решение**


```sql
SELECT
    1000000 AS result
UNION
ALL
SELECT
    541
UNION
ALL
SELECT
    -500
UNION
ALL
SELECT
    100
ORDER BY
    result DESC
LIMIT
    1
```

#### Задание 6.2

>
> Помним, что сортировка для числовых и строковых типов данных отличается. Построив запрос по аналогии с примером, приведите значения к текстовому типу данных, сравните и выберите наибольшее из них:
> 
> * 1000000;
> * 541;
> * -500;
> * 100.
> 
> Столбец с ответом назовите `mycol`
>

**Решение**


```sql
SELECT
    '1000000' AS val
UNION    
SELECT
    '541'
UNION    
SELECT
    '-500'
UNION    
SELECT
    '100'
ORDER BY
    val DESC
LIMIT
    1
```

**Эталонное решение**


```sql
SELECT
    '1000000' as mycol
UNION
ALL
SELECT
    '541'
UNION
ALL
SELECT
    '-500'
UNION
ALL
SELECT
    '100'
ORDER BY
    1 DESC
LIMIT
    1
```

#### Задание 6.3

> Построив запрос по аналогии с примером, найдите самое большое значение из перечисленных операторов:
> 
> * `+` ;
> * `-` ;
> * `=` ;
> * `/` .
> 
> Столбец с ответом назовите `result`.

**Решение**


```sql
SELECT
    '+' AS result
UNION
ALL
SELECT
    '-'
UNION
ALL
SELECT
    '='
UNION
ALL
SELECT
    '/'
ORDER BY
    result DESC
LIMIT
    1
```